In [1]:
import torch


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

device

/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cpu'

In [2]:
usr = "char"
# Tengo GPU pero no cuda compatible, me rompe cuando intento correr con GPU
if usr == "nico":
    device="cpu"
    num_workers = 4
    
if usr == "char":
    num_workers = 0

In [3]:
from catdog.dataset import CatDogDataset

In [4]:
DATA_DIR = '../data/'

In [5]:
import pandas as pd


train_set_pd = pd.read_csv(DATA_DIR + 'train.csv')[:96]
train_set_torch = CatDogDataset(train_set_pd)
print(f"Loaded train set with length {len(train_set_torch)}")

test_set_pd = pd.read_csv(DATA_DIR + 'test.csv')[:64]
test_set_torch = CatDogDataset(test_set_pd)
print(f"Loaded test set with length {len(test_set_torch)}")

Loaded train set with length 96
Loaded test set with length 64


In [6]:
type(test_set_torch[0][1])

numpy.int64

In [7]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set_torch, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set_torch, batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [8]:
import numpy as np


seed = 142
torch.manual_seed(seed)
np.random.seed(seed)

#### test image

import numpy as np
from PIL import Image


test_img_path = DATA_DIR + 'images/' + cat_dog_df["file"].iloc[0]
test_img = Image.open(test_img_path).convert("RGB")

In [9]:
from catdog.models.feedforward import MLPClassifier


model = MLPClassifier(input_size=500*500*3, hidden_sizes=(100, 10))
model.to(device)

/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


MLPClassifier(
  (AUROC): AUROC()
  (model): Sequential(
    (0): Linear(in_features=750000, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=100, out_features=10, bias=True)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): CatDogOutput(
      (classify_layer): Linear(in_features=10, out_features=1, bias=True)
      (bbox_layer): Linear(in_features=10, out_features=4, bias=True)
    )
  )
)

In [10]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(142, workers=True)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='./lightning_checkpoints/',
    filename='test_model_training',
    mode='min',
    save_top_k=3
)

logger = TensorBoardLogger("tb_logs", name="MLP_classifier")

trainer = Trainer(gpus=int(device=='cuda'), # hacky way to say 0 or 1 
                  max_epochs=40,
                  logger=logger,
                  callbacks=[EarlyStopping(monitor="val_loss", patience=15, mode='min'),
                             checkpoint_callback])
trainer.fit(model, train_loader, test_loader)  # here we are using test set as validation set

Global seed set to 142
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type       | Params
-------------------------------------
0 | AUROC | AUROC      | 0     
1 | model | Sequential | 75.0 M
-------------------------------------
75.0 M    Trainable params
0         Non-trainable params
75.0 M    Total params
300.006   Total estimated model params size (MB)


configuring optimizers
Validation sanity check:   0%|                                                                                                                         | 0/2 [00:00<?, ?it/s]

/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


entered validation step
got preds
got loss
Validation sanity check:  50%|████████████████████████████████████████████████████████▌                                                        | 1/2 [00:01<00:01,  1.21s/it]

/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)


entered validation step
got preds
got loss
                                                                                                                                                                             

Global seed set to 142
/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/cgiudice/facu/aprendizaje_automatico/aaut-catdog/.venv/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Epoch 0:  60%|█████████████████████████▏                | 3/5 [00:04<00:02,  1.36s/it, loss=4.29e+04, v_num=8, train_classification_loss=0.639, train_bbox_loss_loss=4.55e+4]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                                                                                      | 0/2 [00:00<?, ?it/s]entered validation step
got preds
got loss

Epoch 0: 100%|██████████████████████████████████████████| 5/5 [00:05<00:00,  1.00s/it, loss=4.29e+04, v_num=8, train_classification_loss=0.639, train_bbox_loss_loss=4.55e+4]entered validation step
got preds
got loss
Epoch 0: 100%|█| 5/5 [00:05<00:00,  1.04s/it, loss=4.29e+04, v_num=8, train_classification_loss=0.639, train_bbox_loss_loss=4.55e+4, validation_classification_loss=7.120, va
Epoch 1:  80%|▊| 4/5 [00:03<00:00,  1.24it/s, loss=4.29e+04, v_num=8, train_classification_loss=0.738, train_bbox_loss_loss=4.49e+4, validation_classification_loss=7.120, va
Validating: 0i

In [ ]:
torch.flatten